<a href="https://colab.research.google.com/github/Enkrumah14/mannyNkrumahGenAi/blob/main/HW4/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#author : Manny Nkrumah
#file : Problem1.ipynb
#assignment # : 4
#date : 11/04/24
#description : GAN architecture bedroom model

!pip install patool
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import os
import glob
from PIL import Image
import time
from tqdm import tqdm
import requests
import patoolib
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Hyperparameters
EPOCHS = 50
BATCH_SIZE = 256
noise_dim = 100
num_examples_to_generate = 16
generator_lr = 1e-4
discriminator_lr = 1e-4

# Links
archive_path = "archive.zip"
output_dir = "dataset"
url = "https://www.kaggle.com/api/v1/datasets/download/jhoward/lsun_bedroom"  # Dataset URL

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Download dataset archive in chunks
with requests.get(url, stream=True) as r:
    r.raise_for_status()  # Check for download errors
    with open(archive_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print(f"Dataset downloaded to: {archive_path}")

# Extract archive using patoolib
try:
    patoolib.extract_archive(archive_path, outdir=output_dir, program="unzip")
except patoolib.util.PatoolError:
    try:
        patoolib.extract_archive(archive_path, outdir=output_dir, program="7z")
    except patoolib.util.PatoolError:
        print("Extraction failed: check if the file is corrupted or in a supported format.")

# Load image file paths
image_files = glob.glob(f"{output_dir}/**/*.jpg", recursive=True)
if not image_files:
    raise ValueError("No image files found. Please check the output_dir path.")
print(f"Found {len(image_files)} image files.")

# Define a function to load and preprocess images
def load_and_preprocess_image(path):
    """Loads an image from the given path and preprocesses it."""
    image = tf.io.read_file(path)  # Read the image file
    image = tf.image.decode_jpeg(image, channels=3)  # Decode JPEG image
    image = tf.image.resize(image, [64, 64])  # Resize to 64x64
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

# Create a TensorFlow dataset and preprocess images
dataset = tf.data.Dataset.from_tensor_slices(image_files)
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)  # Apply the preprocessing function
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)


# Define generator model
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Define discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Loss functions for GAN training
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(generator_lr)
discriminator_optimizer = tf.keras.optimizers.Adam(discriminator_lr)

# Define training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Define training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for image_batch in tqdm(dataset, desc=f"Epoch {epoch+1}/{epochs}"):
            train_step(image_batch)
        generate_and_save_images(generator, epoch + 1, seed)
        print(f'Time for epoch {epoch + 1} is {time.time()-start:.2f} sec')

# Generate and save images during training
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :] * 0.5 + 0.5)
        plt.axis('off')
    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

# Main execution
generator = make_generator_model()
discriminator = make_discriminator_model()
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

train(dataset, EPOCHS)  # Start training


INFO patool: Extracting archive.zip ...
INFO:patool:Extracting archive.zip ...
INFO patool: running /usr/bin/unzip -- archive.zip -d dataset
INFO:patool:running /usr/bin/unzip -- archive.zip -d dataset


Dataset downloaded to: archive.zip


INFO patool: ... archive.zip extracted to `dataset'.
INFO:patool:... archive.zip extracted to `dataset'.


Found 606250 image files.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Epoch 1/10:   5%|▍         | 112/2369 [18:34<6:14:23,  9.95s/it]


KeyboardInterrupt: 

In [ ]:
!pip install patool
!pip install requests
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import time
import os
import glob
import patoolib
import requests
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
generator_lr = 1e-4
discriminator_lr = 1e-4

# Links
archive_path = "archive.zip"
output_dir = "dataset"
url = "https://www.kaggle.com/api/v1/datasets/download/jhoward/lsun_bedroom"  # Dataset URL

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Download dataset archive in chunks
with requests.get(url, stream=True) as r:
    r.raise_for_status()  # Check for download errors
    with open(archive_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print(f"Dataset downloaded to: {archive_path}")

# Extract archive using patoolib
try:
    patoolib.extract_archive(archive_path, outdir=output_dir, program="unzip")
except patoolib.util.PatoolError:
    try:
        patoolib.extract_archive(archive_path, outdir=output_dir, program="7z")
    except patoolib.util.PatoolError:
        print("Extraction failed: check if the file is corrupted or in a supported format.")

# Load image file paths
image_files = glob.glob(f"{output_dir}/**/*.jpg", recursive=True)
if not image_files:
    raise ValueError("No image files found. Please check the output_dir path.")
print(f"Found {len(image_files)} image files.")

# Define a function to load and preprocess images
def load_and_preprocess_image(path):
    """Loads an image from the given path and preprocesses it."""
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [64, 64])
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5
    return image

# Create a TensorFlow dataset and preprocess images
dataset = tf.data.Dataset.from_tensor_slices(image_files)
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)

# Define generator model
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Define discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Loss functions for GAN training
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(generator_lr)
discriminator_optimizer = tf.keras.optimizers.Adam(discriminator_lr)

# Define training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Load pre-trained InceptionV3 model
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

# Implement Inception Score calculation
def inception_score(generator, n_split=10, eps=1E-16):
    n_samples = BATCH_SIZE * n_split
    generated_images = []
    for _ in range(n_split):
        noise = tf.random.normal([BATCH_SIZE, noise_dim])
        imgs = generator(noise, training=False)
        imgs = tf.image.resize(imgs, (299, 299))
        imgs = preprocess_input(imgs * 255)  # Scale to 0-255 and preprocess
        generated_images.append(imgs)
    generated_images = tf.concat(generated_images, axis=0)

    preds = inception_model.predict(generated_images)
    scores = []
    for i in range(n_split):
        part = preds[i * BATCH_SIZE:(i+1) * BATCH_SIZE]
        kl = part * (np.log(part + eps) - np.log(np.expand_dims(np.mean(part, 0), 0) + eps))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))

    return np.mean(scores), np.std(scores)

# Generate and save images during training
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :] * 0.5 + 0.5)
        plt.axis('off')
    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

# Define training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        gen_losses = []
        disc_losses = []
        for image_batch in tqdm(dataset, desc=f"Epoch {epoch+1}/{epochs}"):
            gen_loss, disc_loss = train_step(image_batch)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        print(f'Epoch {epoch + 1}, Gen Loss: {np.mean(gen_losses):.4f}, Disc Loss: {np.mean(disc_losses):.4f}')
        generate_and_save_images(generator, epoch + 1, seed)

        # Calculate and log Inception Score every 10 epochs
        if (epoch + 1) % 10 == 0:
            is_mean, is_std = inception_score(generator)
            print(f'Inception Score - Mean: {is_mean:.3f}, Std: {is_std:.3f}')

        print(f'Time for epoch {epoch + 1} is {time.time()-start:.2f} sec')

# Main execution
generator = make_generator_model()
discriminator = make_discriminator_model()

print("Generator Summary:")
generator.summary()

print("\nDiscriminator Summary:")
discriminator.summary()

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

train(dataset, EPOCHS)  # Start training with improved logging and evaluation

INFO patool: Extracting archive.zip ...
INFO:patool:Extracting archive.zip ...


Dataset downloaded to: archive.zip


INFO patool: running /usr/bin/unzip -- archive.zip -d dataset
INFO:patool:running /usr/bin/unzip -- archive.zip -d dataset
INFO patool: ... archive.zip extracted to `dataset'.
INFO:patool:... archive.zip extracted to `dataset'.


Found 606250 image files.
87910968/87910968 [==============================] - 0s 0us/step
Generator Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16384)             1638400   
                                                                 
 batch_normalization_94 (Ba  (None, 16384)             65536     
 tchNormalization)                                               
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16384)             0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         819200    
 anspose)                                                        
            

Epoch 1/50: 100%|██████████| 9473/9473 [18:05<00:00,  8.73it/s]


...

Epoch 3/50: 100%|██████████| 9473/9473 [18:01<00:00,  8.76it/s]


Epoch 3, Gen Loss: 3.2776, Disc Loss: 0.4919
Time for epoch 3 is 1082.39 sec


Epoch 4/50: 100%|██████████| 9473/9473 [17:52<00:00,  8.83it/s]


Epoch 4, Gen Loss: 3.2051, Disc Loss: 0.5347
Time for epoch 4 is 1073.18 sec


Epoch 5/50:  94%|█████████▎| 8876/9473 [16:56<01:08,  8.71it/s]